In [19]:
#Imports
import cv2
import numpy as np
import glob

In [20]:
#Define checkerboard size
checkerboard_size = (24,17)
frame_size = (1440, 1080)

In [21]:
#termination criteria
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

In [22]:
#Prepare object points
object_points = np.zeros((checkerboard_size[0] * checkerboard_size[1], 3), np.float32)
object_points[:,:2] = np.mgrid[0:checkerboard_size[0], 0:checkerboard_size[1]].T.reshape(-1,2)

In [23]:
#Arrays to store object points and image points from all images
object_points_list = [] #3d points real world space
image_points_list = [] #2d points image plain

In [24]:
#Path to the images
images = glob.glob("/Users/davidschaupp/Documents/GitHub/Vision/Camera_Calibration/Calibration/Checkerboard_Images/*.png")

In [25]:
for image in images:
    img = cv2.imread(image)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    #detect corners
    ret, corners = cv2.findChessboardCorners(gray, checkerboard_size, None)

    if ret == True:
        object_points_list.append(object_points)
        corners_2 = cv2.cornerSubPix(gray, corners, (11, 11), (-1,-1), criteria)
        image_points_list.append(corners)

        #Draw and display the corners
        """
        cv2.drawChessboardCorners(img, checkerboard_size, corners_2, ret)
        cv2.imshow("img", img)
        cv2.waitKey(1000)
        """

cv2.destroyAllWindows()

In [26]:
#Calibration
ret, cameraMatrix, dist, rvecs, tvecs = cv2.calibrateCamera(object_points_list, image_points_list, frame_size, None, None)

In [27]:
print(f"Camera calibrated: {ret}")
print(f"\n Camera Matrix: \n {cameraMatrix}")
print(f"\n Disortion Parameters: \n {dist}")
print(f"\n Rotation Vectors: \n {rvecs}")
print(f"\nTranslation Vectors: \n {tvecs}")

Camera calibrated: 1.8174966752742883

 Camera Matrix: 
 [[1.17298450e+03 0.00000000e+00 7.41150967e+02]
 [0.00000000e+00 1.17330896e+03 5.90933024e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]

 Disortion Parameters: 
 [[-0.24453333  0.1398594   0.00112469 -0.00032557 -0.07670913]]

 Rotation Vectors: 
 [array([[-0.02048924],
       [-0.29762715],
       [-0.06964861]]), array([[ 3.29209757e-01],
       [ 2.09403891e-04],
       [-2.39759025e-02]]), array([[-0.00710631],
       [ 0.44190207],
       [-0.00083659]]), array([[-0.6413161 ],
       [ 0.02735953],
       [-0.05102055]]), array([[-0.00898901],
       [ 0.00851589],
       [ 0.02877699]]), array([[-0.00750662],
       [-0.00023334],
       [ 0.00011609]]), array([[-0.01061286],
       [ 0.00212103],
       [ 0.02078536]]), array([[-0.00388639],
       [-0.00149317],
       [-0.0193396 ]]), array([[-0.01347142],
       [ 0.24388383],
       [-0.00554288]]), array([[-0.0110545 ],
       [-0.00054506],
       [ 0.037868

In [28]:
#Get new camera matrix
img = cv2.imread("/Users/davidschaupp/Documents/GitHub/Vision/Camera_Calibration/Calibration/Checkerboard_Images/Image__2018-10-05__10-29-04.png")
h, w = img.shape[:2]
new_camera_matrix, roi = cv2.getOptimalNewCameraMatrix(cameraMatrix, dist, (w,h), 1, (w,h))

In [29]:
#Undistort
dst = cv2.undistort(img, cameraMatrix, dist, None, new_camera_matrix)
#crop the image
x, y, w, h = roi
dst = dst[y:y+h, x:x+w]
cv2.imwrite("/Users/davidschaupp/Documents/GitHub/Vision/Camera_Calibration/Calibration/Checkerboard_Images/calibration_result.png", dst)

True

In [31]:
#Undistort with remapping
mapx, mapy = cv2.initUndistortRectifyMap(cameraMatrix, dist, None, new_camera_matrix, (w,h), 5)
dst = cv2.remap(img, mapx, mapy, cv2.INTER_LINEAR)
#crop the image
x, y, w, h = roi
dst = dst[y:y+h, x:x+w]
cv2.imwrite("/Users/davidschaupp/Documents/GitHub/Vision/Camera_Calibration/Calibration/Checkerboard_Images/calibration_result_mapped.png", dst)



True